In [1]:
from qdrant_client import QdrantClient, models

/home/berk/finreg-navigator/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = QdrantClient(url="http://localhost:6333")

In [3]:
collection_name = "bank_compliance_v1"

client.create_collection(
    collection_name=collection_name,
    vectors_config={
        "dense": models.VectorParams(
            size=768,
            distance=models.Distance.COSINE,
            hnsw_config=models.HnswConfigDiff(
                m=8,
                ef_construct=100
            )
        )
    },
    sparse_vectors_config={
        "sparse": models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    },
    optimizers_config=models.OptimizersConfigDiff(
        indexing_threshold=1000
    )
)

client.create_payload_index(collection_name, "file_name", models.PayloadSchemaType.KEYWORD)
client.create_payload_index(collection_name, "page_label", models.PayloadSchemaType.INTEGER)
client.create_payload_index(collection_name, "article_no", models.PayloadSchemaType.KEYWORD)

UpdateResult(operation_id=6, status=<UpdateStatus.COMPLETED: 'completed'>)

In [4]:
import os
from llama_index.core import SimpleDirectoryReader

def banking_metadata_extractor(file_path):
    """
    Her dosya okunduğunda dosya bazlı sabit metadata'ları ekler.
    LlamaIndex sayfa numaralarını (page_label) otomatik olarak dökümana ekleyecektir.
    """
    file_name = os.path.basename(file_path)
    
    # Dosya ismine göre döküman başlığı atama
    doc_titles = {
        "BBSEBH.pdf": "Bankaların Bilgi Sistemleri ve Elektronik Bankacılık Hizmetleri Hakkında Yönetmelik",
        "sir_saklama_yukumlulugu.pdf": "Sır Niteliğindeki Bilgilerin Paylaşılması Hakkında Yönetmelik"
    }
    
    return {
        "file_name": file_name,
        "document_title": doc_titles.get(file_name, "Bilinmeyen Mevzuat"),
        "file_path": file_path,
        "category": "Banking Regulation",
        "ingestion_date": "2024-05-22"
    }

# Veriyi Yükleme
reader = SimpleDirectoryReader(
    input_dir="./data", 
    file_metadata=banking_metadata_extractor
)
documents = reader.load_data()

print(documents[0].metadata)

{'page_label': '1', 'file_name': 'BBSEBH.pdf', 'document_title': 'Bankaların Bilgi Sistemleri ve Elektronik Bankacılık Hizmetleri Hakkında Yönetmelik', 'file_path': '/home/berk/finreg-navigator/data/BBSEBH.pdf', 'category': 'Banking Regulation', 'ingestion_date': '2024-05-22'}


In [6]:
from llama_index.core.node_parser import SentenceSplitter

# Hierarchical yerine SentenceSplitter kullanıyoruz. Hukuki metinler için en iyisidir.
node_parser = SentenceSplitter(
    chunk_size=512,
    chunk_overlap=100
)

nodes = node_parser.get_nodes_from_documents(documents)

In [7]:
import re

def enrich_metadata_with_inheritance(nodes):
    """
    Her node'un ebeveyninden madde numarasını miras almasını sağlar.
    """
    node_dict = {n.node_id: n for n in nodes}
    
    for node in nodes:
        match = re.search(r"MADDE\s+(\d+)", node.text)
        if match:
            node.metadata["article_no"] = f"Madde {match.group(1)}"
        else:
            parent_id = node.parent_node.node_id if node.parent_node else None
            while parent_id and "article_no" not in node.metadata:
                parent = node_dict.get(parent_id)
                if parent:
                    parent_match = re.search(r"MADDE\s+(\d+)", parent.text)
                    if parent_match:
                        node.metadata["article_no"] = f"Madde {parent_match.group(1)}"
                        break
                    parent_id = parent.parent_node.node_id if parent.parent_node else None
                else:
                    break
            
            if "article_no" not in node.metadata:
                node.metadata["article_no"] = "Genel Hükümler"
                
    return nodes

enriched_nodes = enrich_metadata_with_inheritance(nodes)

In [ ]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Banking_RAG_Ingestion")

with mlflow.start_run(run_name="Initial_Ingestion_Hybrid"):
    mlflow.log_param("chunk_sizes", "[2048, 512, 128]")
    mlflow.log_param("vector_store", "Qdrant")
    mlflow.log_param("collection_name", "banking_compliance_v1")
    
    mlflow.log_metric("total_nodes", len(enriched_nodes))
    
    print(f"Toplam {len(enriched_nodes)} node oluşturuldu ve MLflow'a loglandı.")

In [8]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# 1. Embedding Modelini Tanımla (Dense)
#embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")

# 2. Qdrant Store Yapılandırması
vector_store = QdrantVectorStore(
    collection_name="bank_compliance_v1",
    client=client,
    enable_hybrid=True,
    batch_size=64,
    dense_vector_name="dense",
    sparse_vector_name="sparse"
)

# 3. Indexleme İşlemi
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(
    enriched_nodes, 
    storage_context=storage_context,
    embed_model=embed_model
)


print("İndeksleme başarıyla tamamlandı!")

2026-02-23 22:46:33,282 - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-base
2026-02-23 22:46:33,581 - INFO - HTTP Request: HEAD https://huggingface.co/intfloat/multilingual-e5-base/resolve/main/modules.json "HTTP/1.1 307 Temporary Redirect"
2026-02-23 22:46:33,623 - INFO - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/intfloat/multilingual-e5-base/835193815a3936a24a0ee7dc9e3d48c1fbb19c55/modules.json "HTTP/1.1 200 OK"
2026-02-23 22:46:33,768 - INFO - HTTP Request: HEAD https://huggingface.co/intfloat/multilingual-e5-base/resolve/main/config_sentence_transformers.json "HTTP/1.1 404 Not Found"
2026-02-23 22:46:33,918 - INFO - HTTP Request: HEAD https://huggingface.co/intfloat/multilingual-e5-base/resolve/main/config_sentence_transformers.json "HTTP/1.1 404 Not Found"
2026-02-23 22:46:34,064 - INFO - HTTP Request: HEAD https://huggingface.co/intfloat/multilingual-e5-base/resolve/main/README.md "HTTP/1.1 307 Temporary Redirect"
2026-02-23 22:46:

İndeksleme başarıyla tamamlandı!


In [18]:
retriever = index.as_retriever(similarity_top_k=3)

# Test Sorgusu
query = "Açık bankacılık servisleri nedir?"
retrieved_nodes = retriever.retrieve(query)

for i, node in enumerate(retrieved_nodes):
    print(f"\n--- Sonuç {i+1} (Skor: {node.score:.4f}) ---")
    print(f"Kaynak: {node.metadata.get('file_name')} | Sayfa: {node.metadata.get('page_label')}")
    print(f"Madde: {node.metadata.get('article_no', 'Belirtilmemiş')}")
    print(f"İçerik Özeti: {node.text[:200]}...")

2026-02-23 22:54:12,153 - INFO - HTTP Request: POST http://localhost:6333/collections/bank_compliance_v1/points/query/batch "HTTP/1.1 200 OK"



--- Sonuç 1 (Skor: 0.8454) ---
Kaynak: BBSEBH.pdf | Sayfa: 15
Madde: Madde 41
İçerik Özeti: c) Ana menüde veya alt menülerde çağrı merkezi görevlisi veya müşteri temsilcisine bağlanma seçeneği sunulur.
ç) Çağrı karşılama hedefinin tutturulması için çağrı merkezi görevlisi veya müşteri temsil...

--- Sonuç 2 (Skor: 0.8427) ---
Kaynak: BBSEBH.pdf | Sayfa: 1
Madde: Madde 1
İçerik Özeti: BANKALARIN BİLGİ SİSTEMLERİ VE ELEKTRONİK BANKACILIK
HİZMETLERİ HAKKINDA YÖNETMELİK
 
BİRİNCİ KISIM
Başlangıç Hükümleri
Amaç ve kapsam
MADDE 1 – 
(1) Bu Yönetmeliğin amacı, bankaların faaliyetlerinin ...

--- Sonuç 3 (Skor: 0.8379) ---
Kaynak: BBSEBH.pdf | Sayfa: 14
Madde: Genel Hükümler
İçerik Özeti: bildirimlerin en kısa sürede giderilmesine yönelik gerekli çalışmaların yapılması sağlanır.
(4) Banka tarafından sunulan elektronik bankacılık hizmetlerinde, müşterilerin yanlış işlem yapma ihtimalini...


In [10]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3", request_timeout=120.0)

#response = llm.complete("Bankacılıkta uyum (compliance) ne anlama gelir?")
#print(response)

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import LLMRerank

base_retriever = index.as_retriever(similarity_top_k=5)

#reranker = LLMRerank(choice_batch_size=5, top_n=5, llm=llm)

# 3. Query Engine oluşturma (LLM + Retriever + Reranker)
query_engine = RetrieverQueryEngine.from_args(
    base_retriever,
    llm=llm,
)

In [20]:
from llama_index.core import PromptTemplate

qa_prompt_tmpl_str = (
    "SADECE SANA VERİLEN BAĞLAMDAKİ BİLGİLERİ KULLANARAK CEVAP VER.\n"
    "KESİNLİKLE TÜRKÇE DIŞINDA BİR DİL KULLANMA.\n"
    "Sen kıdemli bir Bankacılık Uyum (Compliance) Analistisin.\n"
    "Cevabını verirken mutlaka hangi maddeye dayandığını belirt (Örn: Madde 3 uyarınca...).\n"
    "Eğer bağlamda bilgi yoksa, 'Bu bilgi dökümanda yer almamaktadır' de.\n"
    "---------------------\n"
    "BAĞLAM:\n{context_str}\n"
    "---------------------\n"
    "SORU: {query_str}\n"
    "YANIT (SADECE TÜRKÇE): "
)

qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

In [21]:
response = query_engine.query("Açık bankacılık servisleri nedir?")

print(f"Cevap: \n{response}")

for source in response.source_nodes:
    print(f"\nKullanılan Madde: {source.node.metadata.get('article_no')}")
    print(f"Kaynak Dosya: {source.node.metadata.get('file_name')}")

2026-02-23 22:57:42,707 - INFO - HTTP Request: POST http://localhost:6333/collections/bank_compliance_v1/points/query/batch "HTTP/1.1 200 OK"
2026-02-23 22:58:14,150 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Cevap: 
According to Madde 3, Açık bankacılık servisleri are defined as:

"Oğaç bankacılık servisleri: Müşterilerin ya da müşteriler adına hareket eden tarafların API, web servis, dosya transfer protokolü gibi yöntemlerle bankanın sunduğu finansal servislere uzaktan erişerek bankacılık işlemlerini gerçekleştirebildikleri veya gerçekleştirilmesi için bankaya talimat verebildikleri elektronik dağıtım kanalını,"

Translation: "Open banking services: The API, web service, file transfer protocol, etc. methods by which customers or persons acting on behalf of customers can remotely access and conduct financial transactions with a bank's offered services, or receive instructions from the bank to perform such transactions."

Kullanılan Madde: Madde 41
Kaynak Dosya: BBSEBH.pdf

Kullanılan Madde: Madde 1
Kaynak Dosya: BBSEBH.pdf

Kullanılan Madde: Genel Hükümler
Kaynak Dosya: BBSEBH.pdf

Kullanılan Madde: Madde 16
Kaynak Dosya: BBSEBH.pdf

Kullanılan Madde: Madde 43
Kaynak Dosya: BBSEBH.pdf
